In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

from joblib import Parallel,delayed

import pandas as pd

from joblib import Parallel,delayed

from pathlib import Path

from collections import defaultdict

In [2]:
HL_LOWER = 0
HL_UPPER = 0.5474

PL_LOWER = -1.8929
PL_UPPER = 4.6982

In [3]:
data_dir = Path("features")
data_dir1 = data_dir/"MIT-CS1PD"
data_dir2 = data_dir/"MIT-CS2PD"

save_dir = Path("processed_features")
save_dir.mkdir(exist_ok=True)

save_dir1 = save_dir/"MIT-CS1PD"
save_dir1.mkdir(exist_ok=True)

save_dir2 = save_dir/"MIT-CS2PD"
save_dir2.mkdir(exist_ok=True)

In [4]:
def preprocess(sample):
    sample = sample.drop(["Press","Release"],axis=1)
    sample = sample[
        np.logical_and(HL_LOWER<sample["HL"], sample["HL"]<HL_UPPER)
    ].reset_index(drop=True)
    
    sample.loc[
        ~np.logical_and(
            PL_LOWER<sample["PL"],
            sample["PL"]<PL_UPPER
        ),
        ["IL","PL","RL","CL"]
    ] = np.nan

    sample = sample.astype(
        {"HL":"float32","IL":"float32","PL":"float32","RL":"float32","CL":"float32",}
        )
    
    return sample

In [5]:
def compute_diff(sample):
    for key in ["HL","IL","PL","RL","CL"]:
        diff = sample[key][1:].values-sample[key][:-1].values
        sample[f"diff_{key}"] = np.concatenate(
            [
                [np.nan],diff
            ]
        )
    return sample

In [6]:
def pipeline(sample_path,save_dir):
    sample = pd.read_csv(sample_path)
    sample = preprocess(sample)
    sample = compute_diff(sample)
    sample.to_csv(save_dir/sample_path.name,index=False)

In [7]:
Parallel(n_jobs=-2)(
    delayed(pipeline)(sample_path,save_dir1) for sample_path in data_dir1.glob("*.csv")
);

In [8]:
Parallel(n_jobs=-2)(
    delayed(pipeline)(sample_path,save_dir2) for sample_path in data_dir2.glob("*.csv")
);